In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [8]:
from google.colab import files

uploaded = files.upload()

Saving atis-2.dev.iob to atis-2.dev.iob
Saving atis-2.dev.w-intent.iob to atis-2.dev.w-intent.iob
Saving atis-2.train.iob to atis-2.train.iob
Saving atis-2.train.w-intent.iob to atis-2.train.w-intent (1).iob
Saving atis.test.iob to atis.test.iob
Saving atis.test.w-intent.iob to atis.test.w-intent.iob
Saving atis.train.iob to atis.train.iob
Saving atis.train.w-intent.iob to atis.train.w-intent.iob
Saving output1.csv to output1.csv
Saving processed_train_data.pkl to processed_train_data.pkl
Saving sample.iob to sample.iob


##**Sequence to Sequence Learning**






Sequence to Sequence Network or Encoder Decoder network can be defined as the 
training of model which have the capability to convert sequence from one domain 
to sequence in another domain. Input and output squence can be or cannot be of same length. This architecture has many applications such as machine translation, question answering task,image captioning and speech recognization. Intent detection and slot filling falling under the domain of natural language understanding has seen great sucess after the use of this architecture. In this project we are working on this problem area.

This model utilizes two RNNs Encoder and decoder. Input sequence is fed to the encoder which generates an output vector. Final output from encoder is called as a context vector which acts as an input to the decoder, which generates output sequence. 

Though Deep Neural Network are extremely powerful models but they have a limitation of being application to problems where input and output vectors can be encoded with fixed dimensionality. But in Sequence to Sequence model, inputs are encoded in to one context vector and decoder produces output from this one context vector to many thus addressing the limitation of one to one relationship between input and output.

 ![alt text](https://drive.google.com/uc?id=11v1h9NIHJcHf4H-7t6ujTnJRJLU-96Ez)

##**Attention Mechanism**



Use of Attention enhances the performance of Encoder Decoder Architecture. In above stated architecture, input sequence is compressed into one context vector which represents the entire input sequence. It has been seen that due to this performance of architecture declines if input sequence is of long length as for example two sentences of large length and with similar words are input to model, then encoder and decoder should be efficient enough to determine the small differences in sequences. Use of Attention address this problem as when decoder predicts the output it uses part of a sentence where the relevant information is present rather than the entire sentence. By introducing the attention, encoder remains same but changes are made in decoder part. In decoder, another Feed forward layer will be introduced to which current input and hidden state are fed which produces new vector which is passed through the softmax to create weights which are multiplied with previous context vector generated by encoder to create new context vector. This context vector is then used to predict next output.

![alt text](https://drive.google.com/uc?id=1cF5x4PhGgLD_Ja5amEmWR6kUlgJNbQ1j)




### Requirements


In [0]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

import pickle
import random
import os
import pdb
from torch.autograd import Variable
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

import torch.optim as optim
import argparse
import numpy as np



In [0]:

USE_CUDA = torch.cuda.is_available()

Dataset - ATIS

Atis dataset is a benchmark dataset used in natural language understanding domain. Training dataset contains 4478 recordings where each line consists of conversations,slot tags and intent values. There are 21 intent categories and 120 distinct slot tags. We defined the vocabulary by extracting the unique values from the conversation and defined the maximum length as 50 for slot and conversation tokens and  padded  using  pad  tag  if  length  of  sentence is less than 50.  Later we created three dictionaries for words in vocabulary, intent and slot tags such that token of each category would be key and value would its position.  

In order to encode the categorical data we used the label-encoding function provided by the SciKit learn library. This encoding is done as we'll need a unique index per word to use as the inputs and targets of the networks. We performed encoding on train data while training and prepared the true intent and true slot files too in order to use as true reference while producing the final accuracy and f1 score.

In the following cell, def pad() function is used to bring uniformity in the sequence of varying length input senetences, therefore by using the pad symbol shorter sentences will be made in length equal to all sentences.  All the sequences are appended with the EOS tag in order to mark the end of sequence. 

Since we are using neural network, therefore input fed to neural netwoek should be something that it can understand. Therefore, we used preparesequence() function to turn the words into LongTensors. 


In [0]:

flatten = lambda l: [item for sublist in l for item in sublist]

EOS = '<EOS>'
UNK = '<UNK>'
PAD = '<PAD>'
SOS = '<SOS>'


# this step performs the padding of sentence
def pad(some_sequence, length=60):
    if len(some_sequence) < length:
        while len(some_sequence) < length:
            some_sequence.append(PAD)
    else:
        some_sequence = some_sequence[:length]
        some_sequence[-1] = EOS

    return some_sequence

# this function converts sequence to label
def prepare_sequence(seq, to_ix):
    idxs = list(map(lambda w: to_ix[w] if w in to_ix.keys() else to_ix[UNK], seq))
    
    tensor = Variable(torch.LongTensor(idxs))
    # print(tensor)
    # tensor = tensor.view(1, -1)

    return tensor


def labeler_to_ix(le):
    return dict(zip(le.classes_, le.transform(le.classes_))) 

# this function separates the tab separated files into three categories.
def get_raw(file_path):
    train = open(file_path, "r").readlines()
    train = [t[:-1] for t in train]
    train = [[t.split("\t")[0].split(" "), t.split("\t")[1].split(" ")[:-1], t.split("\t")[1].split(" ")[-1]] for t in train]
    train = [[t[0][1:-1], t[1][1:], t[2]] for t in train]

    return train

The full process of preparing train for training consists of following steps:

1.  Read file, split line into three categories.
2.  Perform the padding and in order to make alignment between the train and truth same. 
3. Perform the label encoding.

Following function returns the word2index, intent2index,tag2index

Here, we are using GPU therefore, we have prepared batches of sequences to train at once in order to utilize the GPU efficiently.





In [0]:
def preprocessing(file_path, length):
    """
    atis-2.train.w-intent.iob
    """
    # processed_path = os.path.join(os.path.dirname(os.path.abspath(__file__)), "data/")
    processed_path=file_path
    print("processed_data_path : %s" % processed_path)

    if os.path.exists(os.path.join( "processed_train_data.pkl")):
        train_data, word2index, tag2index, intent2index = pickle.load(open(os.path.join(
                                                                                        "processed_train_data.pkl"),
                                                                           "rb"))
        return train_data, word2index, tag2index, intent2index
                                  
    if not os.path.exists(processed_path):
        os.makedirs(processed_path)
    
    try:
        train = open(file_path, "r").readlines()
        print("Successfully load data. # of set : %d " % len(train))
    except:
        print("No such file!")
        return None, None, None, None

    try:
        vocab = set()
        slot_tag = set()
        intent_tag = set()

        padded_sentences = []
        padded_slots = []
        all_intents = []

        for i in range(len(train)):
           
            bunch_of_lines = train[i]

            
            line = bunch_of_[:-1]

            tab_split = line.split("\t")
            words_in_sentence = tab_split[0].split(" ")[1:-1]
            slots_in_sentence = tab_split[1].split(" ")[:-1][1:]
            intent = tab_split[1].split(" ")[-1]

            assert len(words_in_sentence) == len(slots_in_sentence)
            assert len(intent) > 0

            vocab.update(words_in_sentence)
            slot_tag.update(slots_in_sentence)

            intent_tag.update([intent])

            if len(words_in_sentence) < length:
                words_in_sentence.append(EOS)
            
            padded_sentences.append(pad(words_in_sentence))
            padded_slots.append(pad(slots_in_sentence))
            all_intents.append(intent)

        print("# of vocab : {vocab}, # of slot_tag : {slot_tag}, # of intent_tag : {intent_tag}".format(vocab=len(vocab), slot_tag=len(slot_tag), intent_tag=len(intent_tag)))

        vocab_labenc = LabelEncoder()
        slot_labenc = LabelEncoder()
        intent_labenc = LabelEncoder()
        vocab_labenc.fit([*vocab, PAD, UNK, SOS, EOS])
        slot_labenc.fit([*slot_tag, PAD, UNK])
        intent_labenc.fit([*intent_tag, UNK])

        train = list(zip(padded_sentences, padded_slots, all_intents))
                
        train_data = []
        word2index = labeler_to_ix(vocab_labenc)
        tag2index = labeler_to_ix(slot_labenc)
        intent2index = labeler_to_ix(intent_labenc)

        for tr in train:
            temp = prepare_sequence(tr[0], word2index)
            temp = temp.view(1, -1)

            temp2 = prepare_sequence(tr[1], tag2index)
            temp2 = temp2.view(1, -1)

            as_idx = intent_labenc.transform([tr[2]])
            temp3 = Variable(torch.LongTensor(as_idx))
            # pdb.set_trace()

            train_data.append((temp, temp2, temp3))

        pickle.dump((train_data,word2index,tag2index,intent2index),open(os.path.join(processed_path, "processed_train_data.pkl"), "wb"))
        # pickle
        print("Preprocessing complete!")
        return train_data, word2index, tag2index, intent2index
        # return train_data, vocab_labenc, slot_labenc, intent_labenc

    except Exception as e:
        print(e)
        return None, None, None, None  


def getBatch(batch_size, train_data):
    random.shuffle(train_data)
    sindex = 0
    eindex = batch_size
    while eindex < len(train_data):
        batch = train_data[sindex:eindex]
        temp = eindex
        eindex = eindex+batch_size
        sindex = temp
        
        yield batch  

In [13]:
preprocessing('atis-2.train.w-intent.iob', 60)

processed_data_path : atis-2.train.w-intent.iob


([(tensor([[428, 828, 781, 382, 395, 216, 781, 298, 665, 798, 148, 149, 149, 149,
            149, 149, 149, 149, 149, 149, 149, 149, 149, 149, 149, 149, 149, 149,
            149, 149, 149, 149, 149, 149, 149, 149, 149, 149, 149, 149, 149, 149,
            149, 149, 149, 149, 149, 149, 149, 149, 149, 149, 149, 149, 149, 149,
            149, 149, 149, 149]]),
   tensor([[121, 121, 121, 121, 121,  47, 121,  76,  65, 113,   0,   0,   0,   0,
              0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
              0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
              0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
              0,   0,   0,   0]]),
   tensor([12])),
  (tensor([[665, 798, 367, 395, 216, 781, 616, 485, 753,   4, 333, 665, 798, 367,
            395, 309, 781, 616, 485, 753,   4, 333, 665, 798, 367, 395, 619, 781,
            616, 485, 753,   4, 333, 148, 149, 149, 149, 149, 149, 149, 149, 149,
          

In [14]:
USE_CUDA

True

## Encoder Architecture

In the encoder architecture, RNN reads the input sequence in forward and backward direction. One RNN is used to read the sequence in forward direction thus producing a hidden state say, h1 and another RNN reads the sequence in reverse order thus generating a hidden state h2. Final hidden state is the concatenation of both these states. Final hidden state carries the information about the entire sequence.

Embedding layer nn.Embedding is used to create embedding for each sequence. Then this embedding is resized and fed to the GRU layer called as nn.GRU which then returns output sequence which will be fed to decoder.

![alt text](https://drive.google.com/uc?id=1nEfFpmFt-RI1eTUxvvG2HOUh-UPrAiGv)



In [0]:
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, batch_size=4, n_layers=1):
        super(Encoder, self).__init__()
        
        self.input_size = input_size
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.batch_size = batch_size
        
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.gru = nn.GRU(embedding_size, hidden_size, n_layers, batch_first=True, bidirectional=True)

    def init_weights(self):
        self.embedding.weight.data.uniform_(-0.1, 0.1)

    def init_hidden(self, input):
        hidden = Variable(torch.zeros(self.n_layers*2, input.size(0), self.hidden_size))

        if USE_CUDA:
            hidden = hidden.cuda()

        return hidden

    def forward(self, input, input_masking):
        """
        input : B,T
        input_masking : B,T
        output : B,T,D  B,1,D
        """

        if USE_CUDA:
            input = input.cuda()


        hidden = self.init_hidden(input)
        
        # B,T,D
        embedded = self.embedding(input).cuda()
        if USE_CUDA:
            embedded = embedded.cuda()
        # B,T,D
        output, hidden = self.gru(embedded.cuda(), hidden)

        real_context = []
        
        for i, o in enumerate(output):
            # B,T,D
            real_length = input_masking[i].data.tolist().count(0)
            real_context.append(o[real_length-1])
        
        return output, torch.cat(real_context).view(input.size(0), -1).unsqueeze(1)

## Decorder Architecture with Attention


The last hidden state of Encoder contains information about the entire sequence of the input, which is fed to decoder. In this architecture, attention is used. Purpose of attention is to preserve the information which might get lost along the forward and backward propagation of long sequences thus now while making a prediction we will not only consume the hidden state but also context vector. In this architecture context vector is calculated as the weighted average of the hidden states.

In [0]:
class Decoder(nn.Module):
    
    def __init__(self, slot_size, intent_size, embedding_size, hidden_size, batch_size=8, n_layers=1, dropout_p=0.1):
        super(Decoder, self).__init__()
        
        self.hidden_size = hidden_size
        self.slot_size = slot_size
        self.intent_size = intent_size
        self.n_layers = n_layers
        self.dropout_p = dropout_p
        self.embedding_size = embedding_size
        self.batch_size = batch_size

        # Define the layers
        self.embedding = nn.Embedding(self.slot_size, self.embedding_size).cuda()

        self.gru = nn.GRU(self.embedding_size+self.hidden_size*2, self.hidden_size, self.n_layers, batch_first=True)
        self.attn = nn.Linear(self.hidden_size, self.hidden_size).cuda()
        self.slot_out = nn.Linear(self.hidden_size*2, self.slot_size).cuda()
        self.intent_out = nn.Linear(self.hidden_size*2, self.intent_size).cuda()
    
    def init_weights(self):
        self.embedding.weight.data.uniform_(-0.1, 0.1)

    def Attention(self, hidden, encoder_outputs, encoder_maskings):
        """
        hidden : 1,B,D
        encoder_outputs : B,T,D
        encoder_maskings : B,T
        output : B,1,D
        """
        # (1,B,D) -> (B,D,1)
        hidden = hidden.squeeze(0).unsqueeze(2)
        # B
        batch_size = encoder_outputs.size(0)
        # T
        max_len = encoder_outputs.size(1)
        # (B*T,D) -> (B*T,D)
        energies = self.attn(encoder_outputs.contiguous().view(batch_size * max_len, -1))
        # (B*T,D) -> B,T,D
        energies = energies.view(batch_size, max_len, -1)
        # (B,T,D) * (B,D,1) -> (B,1,T)
        attn_energies = energies.bmm(hidden).transpose(1, 2)
        # PAD masking
        attn_energies = attn_energies.squeeze(1).masked_fill(encoder_maskings, -1e12)
        
        # B,T
        alpha = F.softmax(attn_energies, dim=1)
        # B,1,T
        alpha = alpha.unsqueeze(1)
        # B,1,T * B,T,D => B,1,D
        context = alpha.bmm(encoder_outputs)
        
        return context
    
    def init_hidden(self, input):
        hidden = Variable(torch.zeros(self.n_layers, input.size(0), self.hidden_size))

        if USE_CUDA:
            hidden = hidden.cuda()

        return hidden
    
    def forward(self, input, context, encoder_outputs, encoder_maskings, training=True):
        """
        input : B,1
        context : B,1,D
        encoder_outputs : B,T,D
        output: B*T,slot_size  B,D
        """
        # B,1 -> B,1,D
        embedded = self.embedding(input.cuda())
        hidden = self.init_hidden(input)
        decode = []
        # B,T,D -> T,B,D
        aligns = encoder_outputs.transpose(0, 1)
        # T
        length = encoder_outputs.size(1)

        for i in range(length):
            # B,D -> B,1,D
            aligned = aligns[i].unsqueeze(1)
            _, hidden = self.gru(torch.cat((embedded, context, aligned), 2), hidden)

            # for Intent Detection
            if i == 0:
                # 1,B,D
                intent_hidden = hidden.clone()
                # B,1,D
                intent_context = self.Attention(intent_hidden, encoder_outputs, encoder_maskings)

                # 1,B,D
                concated = torch.cat((intent_hidden, intent_context.transpose(0, 1)), 2)
                # B,D
                intent_score = self.intent_out(concated.squeeze(0))

            # 1,B,D -> 1,B,2*D
            concated = torch.cat((hidden, context.transpose(0, 1)), 2)
            # B,slot_size
            score = self.slot_out(concated.squeeze(0))

            softmaxed = F.log_softmax(score, dim=1)
            decode.append(softmaxed)
            # B
            _, input = torch.max(softmaxed, 1)
            # B,1 -> B,1,D
            embedded = self.embedding(input.unsqueeze(1)).cuda()
            # B,1,D
            context = self.Attention(hidden, encoder_outputs, encoder_maskings)

        # B,slot_size*T
        slot_scores = torch.cat(decode, 1)

        # B*T,slot_size  B,D
        return slot_scores.view(input.size(0)*length, -1), intent_score

## Results 

In [0]:
from sklearn.metrics import f1_score

def eval(pred_intent,pred_slot,true_intent,true_slot):
  predictedintent = []
  predictedslots = []
  trueslots = []
  trueintent = []

  with open(pred_intent) as file_in:
      for line in file_in:
          predictedintent.append(int(line))

  with open(pred_slot) as file_in:
      for line in file_in:
          predictedslots.append(int(line))

  with open(true_intent) as file_in:
      for line in file_in:
          trueintent.append(int(line))

  with open(true_slot) as file_in:
      for line in file_in:
          trueslots.append(int(line))

  print("Result on slot filling")
  corrects = 0
  for i in range(len(trueslots)):
      if trueslots[i] == predictedslots[i]:
          corrects = corrects + 1
  acc = corrects/len(trueslots)
  print("accuracy: "+str(acc))
  f1 = f1_score(trueslots, predictedslots, average='weighted')
  print("F1 on slot filling: "+str(f1))
  # print("F1 on intent: "+str(f1))
  print("Result on intent classification")
  corrects = 0
  for i in range(len(trueintent)):
      if trueintent[i] == predictedintent[i]:
          corrects = corrects + 1
  acc = corrects/len(trueintent)
  print("accuracy: "+str(acc))
  f1 = f1_score(trueintent, predictedintent, average='weighted')
  print("F1 on intent: "+str(f1))

## Model 1 : LSTM with one Layer

In this model we have used LSTM as basic RNN unit and other parameters are as follows

1.  Number of layers: 1
2.  Learning rate:0.0006
3.  batch size: 16
4.  Rnn unit: LSTM
5. Number of Epochs:8






In [16]:

from google.colab import files
uploaded = files.upload()


Saving trueintent.txt to trueintent (1).txt
Saving trueslots.txt to trueslots (1).txt


In [16]:
from google.colab import files
uploaded = files.upload()

Saving predictedintent.txt to predictedintent (5).txt
Saving predictedslots.txt to predictedslots (5).txt


In [30]:
eval("predictedintent (5).txt","predictedslots (5).txt","trueintent.txt","trueslots.txt")

Result on slot filling
accuracy: 0.7613536735051727
F1 on slot filling: 0.7371040487244951
Result on intent classification
accuracy: 0.848
F1 on intent: 0.8332563742318287


## Plots: 
![alt text](https://drive.google.com/uc?id=1szTXEpYpC5YE_PDEr1w7AUg_tuek9frQ) ![alt text](https://drive.google.com/uc?id=11UzUt82zCOT5GhS9lvFyyGV-UEUnTjA_)


## Model 2 : LSTM with two Layers

In this model we have used LSTM as basic RNN unit and other parameters are as follows

1.  Number of layers: 2
2.  Learning rate: 0.0006
3.  batch size: 8
4.  Rnn unit: LSTM
5. Number of Epochs:8

In [18]:
from google.colab import files
uploaded = files.upload()

Saving predictedintent.txt to predictedintent (6).txt
Saving predictedslots.txt to predictedslots (6).txt


In [29]:
eval("predictedintent (6).txt","predictedslots (6).txt","trueintent.txt","trueslots.txt")

Result on slot filling
accuracy: 0.8060669822900228
F1 on slot filling: 0.7791640533556915
Result on intent classification
accuracy: 0.912
F1 on intent: 0.8950452496099556


## Plots

![alt text](https://drive.google.com/uc?id=1J5lCJx_WSffLmoI_AF2X0a2LE86UL08i)
![alt text](https://drive.google.com/uc?id=1CqGhNiXvyk6DDXnJado4M8L5QQuczlig)

## Model 3 : GRU model

In this model we have used GRU as basic RNN unit and other parameters are as follows

1.  Number of layers: 1
2.  Learning rate: 0.0006
3.  batch size: 16
4.  Rnn unit: GRU
5. Number of Epochs: 8

In [20]:
from google.colab import files
uploaded = files.upload()

Saving predictedintent.txt to predictedintent (7).txt
Saving predictedslots.txt to predictedslots (7).txt


In [28]:
eval("predictedintent (7).txt","predictedslots (7).txt","trueintent.txt","trueslots.txt")

Result on slot filling
accuracy: 0.8951429072418026
F1 on slot filling: 0.8875737382109097
Result on intent classification
accuracy: 0.878
F1 on intent: 0.8713601339248321


## Plots
![alt text](https://drive.google.com/uc?id=1gEk2ihhymkPTTuHDR3Hl9z_qFIhQNJtB)
![alt text](https://drive.google.com/uc?id=1wdwt3sqjkNsDJOlrLo6MlXCCLOVnZWoW)

### Hyperparameter Tuning





While improving the accuracy of intent and slot we observed that improvement in the accuracy of intent resulted in decrease in the accuracy of intent therefore we tried to distribute the weight of loss in given ratios. we experimented with three scenarios and observed the best result with 0.7 and 0.3 distribution.

Case 1.   weight loss distribution between loss1 and loss2 : 0.65 and 0.35

1.   epochs =8
2.   batch_size = 16
3.   learning_rate =0.0006
 









In [31]:
from google.colab import files
uploaded = files.upload()

Saving predictedintent.txt to predictedintent (10).txt
Saving predictedslots.txt to predictedslots (10).txt


In [32]:
eval("predictedintent (10).txt","predictedslots (10).txt","trueintent.txt","trueslots.txt")

Result on slot filling
accuracy: 0.8348237769594951
F1 on slot filling: 0.8249498739334484
Result on intent classification
accuracy: 0.856
F1 on intent: 0.8569944609841224


### Plots

![alt text](https://drive.google.com/uc?id=1I8ipeVAuxicsjk3a-1qsrlHp97h8QGCg)
![alt text](https://drive.google.com/uc?id=150CS0UwfH035O8dtOHWtSARHb5I35t8V)

### Hyperparameter Tuning 
Case 2.   weight loss distribution between loss1 and loss2 : 0.65 and 0.35

1.   epochs =8
2.   batch_size = 16
3.   learning_rate =0.0006



In [33]:
from google.colab import files
uploaded = files.upload()

Saving predictedintent.txt to predictedintent (11).txt
Saving predictedslots.txt to predictedslots (11).txt


In [34]:
eval("predictedintent (11).txt","predictedslots (11).txt","trueintent.txt","trueslots.txt")

Result on slot filling
accuracy: 0.860775030685604
F1 on slot filling: 0.8356698939273053
Result on intent classification
accuracy: 0.878
F1 on intent: 0.8566874190039664


## Plots 
![alt text](https://drive.google.com/uc?id=14FUxegHIpGlLqJ0KR9cWdXwwTj-8yEQe)
![alt text](https://drive.google.com/uc?id=1uaeW6JEizvyYVeNjCDAxb1bfNWmjOWAf)

### Hyperparameter Tuning 
Case 3.   weight loss distribution between loss1 and loss2 : 0.70 and 0.30


1.   epochs =8
2.   batch_size = 16
3.   learning_rate =0.0006

In this case we obtained the best results.

In [35]:
from google.colab import files
uploaded = files.upload()

Saving predictedintent (1).txt to predictedintent (1) (1).txt
Saving predictedslots (1).txt to predictedslots (1) (1).txt


In [36]:
eval("predictedintent (1) (1).txt","predictedslots (1) (1).txt","trueintent.txt","trueslots.txt")

Result on slot filling
accuracy: 0.8674381904260915
F1 on slot filling: 0.8464587919764208
Result on intent classification
accuracy: 0.9
F1 on intent: 0.8830732278870191


### Plots
![alt text](https://drive.google.com/uc?id=1DuQh-mSectOYKjDb3MZT2gVLZZULGabR)
![alt text](https://drive.google.com/uc?id=1CzjXBRo_3slIBk3_LeS_vRr02xXOyUq9)


### Experiment : 7
As above case improved the results slightly, we tried to change the hyperparameters more as follows:
We kept the loss weighted ratio as 0.7 and 0.3.
We decreased the Batch size to 8 and kept the epochs same as 16 in expectation to improve the accuracy as in small batch model has a capability to learn better. 
Accuracy improved slightly. In this case Basic RNN unit was LSTM. 


In [37]:
from google.colab import files
uploaded = files.upload()

Saving predictedintent.txt to predictedintent (12).txt
Saving predictedslots.txt to predictedslots (12).txt


In [38]:
eval("predictedintent (12).txt","predictedslots (12).txt","trueintent.txt","trueslots.txt")

Result on slot filling
accuracy: 0.8786603541995441
F1 on slot filling: 0.8664901557514639
Result on intent classification
accuracy: 0.906
F1 on intent: 0.9005548159264637


### Plots
![alt text](https://drive.google.com/uc?id=1VtamNdTqDtkDfvTM18vrelHQ0U9_rgOe)
![alt text](https://drive.google.com/uc?id=1ruJyb7N5bwwbpFi5Wcft55eJiTkxNAAw)

### Experiment 8

In this experiment we took the GRU Model. Epochs were 16.  Reduction in batch size improved the score highly.


1.   We did this in order to decide the basic RNN unit. We chose GRU as by decreasing the batch size to 8 we observed high increase in both accuracy and F1 Score. From this point now, GRU is used in all experiments.





In [40]:
from google.colab import files
uploaded = files.upload()

Saving predictedintent.txt to predictedintent (14).txt
Saving predictedslots.txt to predictedslots (14).txt


In [41]:
eval("predictedintent (14).txt","predictedslots (14).txt","trueintent.txt","trueslots.txt")

Result on slot filling
accuracy: 0.9733473610380502
F1 on slot filling: 0.9722284667825236
Result on intent classification
accuracy: 0.952
F1 on intent: 0.9505502045068722


### Plots
![alt text](https://drive.google.com/uc?id=1Oc1hvko5Jqk0TLZn2RhRFLgp37TEzgbU)
![alt text](https://drive.google.com/uc?id=154rkONm2K6yhXgyAHIa-S950pZvr9LEk)

### Experiment 9

As scored obtained in Experiment 8 were the highest therefore, we decided to see the results on 8 epochs such that if results do not change much we will save the computation power but accuracy decreased. .

In [42]:
from google.colab import files
uploaded = files.upload()

Saving predictedintent.txt to predictedintent (15).txt
Saving predictedslots.txt to predictedslots (15).txt


In [43]:
eval("predictedintent (15).txt","predictedslots (15).txt","trueintent.txt","trueslots.txt")

Result on slot filling
accuracy: 0.9466947220761003
F1 on slot filling: 0.9423177448932558
Result on intent classification
accuracy: 0.912
F1 on intent: 0.9099346512713665


### Plots

![alt text](https://drive.google.com/uc?id=1NrHpfsd7MMaX4J-FixVmTh1XYDPcVirq)
![alt text](https://drive.google.com/uc?id=1V7gEEKQt3_qTTf5sIwj3LvBm5hiQ35wQ)


## Experiment 10

In this experiment we used 16 epochs with 8 batch size and weighted loss ratio of 0.7 and 0.3 and we observed increse in the score.

In [44]:
from google.colab import files
uploaded = files.upload()

Saving predictedintent (1).txt to predictedintent (1) (2).txt
Saving predictedslots (1).txt to predictedslots (1) (2).txt


In [45]:
eval("predictedintent (1) (2).txt","predictedslots (1) (2).txt","trueintent.txt","trueslots.txt")

Result on slot filling
accuracy: 0.9707171664036472
F1 on slot filling: 0.970795628181691
Result on intent classification
accuracy: 0.956
F1 on intent: 0.9566679653679653


### Plots
![alt text](https://drive.google.com/uc?id=1f8C-Ercx2_NlzhfLetw1dB2czJkJSZyT)
![alt text](https://drive.google.com/uc?id=1JyBmJ99WYGpzeU1rPeHrp3RFpH9adRrX)

## Experiment 11
 This experiment is slight modification of Experiment 7. Here we have decreased the number of epochs to 8 to observe the affect and found that accuracy decreased slightly.

In [27]:
from google.colab import files
uploaded = files.upload()

Saving predictedintent (3).txt to predictedintent (3) (1).txt
Saving predictedslots (2).txt to predictedslots (2) (1).txt


In [28]:
eval("predictedintent (3) (1).txt","predictedslots (2) (1).txt","trueintent (1).txt","trueslots (1).txt")

Result on slot filling
accuracy: 0.9219708925127126
F1 on slot filling: 0.9049625924368505
Result on intent classification
accuracy: 0.882
F1 on intent: 0.8664743831914286


## Plots 
![alt text](https://drive.google.com/uc?id=1b9soUw4kppHwnM_-phNoOfjsPod04etz)
![alt text](https://drive.google.com/uc?id=1RzoZmvA4SNY7mKxJHb13_dRypMb0FFkX)

### Experiment 12

In this case we changed the learning rate to 0.0003 while keeping the batch size 8 and epochs 20. This resulted in decrease in F1 score and accuracy in all cases.


In [10]:
from google.colab import files
uploaded = files.upload()

Saving predictedintent (1).txt to predictedintent (1).txt
Saving predictedslots (1).txt to predictedslots (1).txt


In [12]:
eval("predictedintent (1).txt","predictedslots (1).txt","trueintent.txt","trueslots.txt")

Result on slot filling
accuracy: 0.951779765035946
F1 on slot filling: 0.9464879314680973
Result on intent classification
accuracy: 0.906
F1 on intent: 0.89545042770833


### Plots
![alt text](https://drive.google.com/uc?id=1IYyK3UND2q1Eiz7rdiaG35OJWDiq-tP_)
![alt text](https://drive.google.com/uc?id=11_UwN67smTskT-P5r4Qa526JHP6ydHUS)

### Experiment 13

In this case we have experimented with epochs by reducing them to 8 in weighted average case and of bactch size 8. We observed that though decrease in batch size increases in accuracy but here due to reduction in epochs, accuracy and F1 score does not improve highly.

In [2]:
from google.colab import files
uploaded = files.upload()

Saving predictedintent (3).txt to predictedintent (3).txt
Saving predictedslots (2).txt to predictedslots (2).txt


In [8]:
eval("predictedintent (3).txt","predictedslots (2).txt","trueintent.txt","trueslots.txt")

Result on slot filling
accuracy: 0.9219708925127126
F1 on slot filling: 0.9049625924368505
Result on intent classification
accuracy: 0.882
F1 on intent: 0.8664743831914286


### Plots
![alt text](https://drive.google.com/uc?id=1b9soUw4kppHwnM_-phNoOfjsPod04etz)
![alt text](https://drive.google.com/uc?id=1RzoZmvA4SNY7mKxJHb13_dRypMb0FFkX)

### Experiment 14
 Following the above experiment we changed the learning rate to 0.0006 in order to oberve the affect on learning while keeping the other parameters same. This resulted in increase of accuracy and F1 score. 





In [49]:
from google.colab import files
uploaded = files.upload()

Saving predictedintent (2).txt to predictedintent (2) (1).txt
Saving predictedslots (1).txt to predictedslots (1) (4).txt


In [50]:
eval("predictedintent (2) (1).txt","predictedslots (1) (4).txt","trueintent.txt","trueslots.txt")

Result on slot filling
accuracy: 0.967034893915483
F1 on slot filling: 0.966168344398518
Result on intent classification
accuracy: 0.942
F1 on intent: 0.946525023191208


### Plots
![alt text](https://drive.google.com/uc?id=1E-izL9lKg-ds8RV-suo4nwfC3_MUH-wx)
![alt text](https://drive.google.com/uc?id=12At6iApiP77rLWRogwIL6ghy6nt672UT)

## Experiment 15

In this case we performed the weighted loss on case 11 by distributing the loss between two cases by 0.7 and 0.3 . This resulted in slighter better accuracy of all cases.


In [51]:
from google.colab import files
uploaded = files.upload()

Saving predictedintent (2).txt to predictedintent (2) (2).txt
Saving predictedslots (1).txt to predictedslots (1) (5).txt


In [52]:
eval("predictedintent (2) (2).txt","predictedslots (1) (5).txt","trueintent.txt","trueslots.txt")

Result on slot filling
accuracy: 0.9649307382079607
F1 on slot filling: 0.9636103085714558
Result on intent classification
accuracy: 0.934
F1 on intent: 0.9341634521626107


### Plots
![alt text](https://drive.google.com/uc?id=1ld8RPAOw45_sWxLFx02MepWfdbrzbh_v)
![alt text](https://drive.google.com/uc?id=19mVVX8usSDhPzU2TH_wk_GT5haw7jl80)

## Experiment 16 - Improvement over baseline

To this point we have done testing on dev set and have not used test dataset in order to prevent the generalization of data. In this case we have used 2 layered GRU unit and observed the results.

This gave us improvement over F1 score of slot Filling, paper stated F1 score of 95.75 and we obtained 96.08.



Results on Dev set

In [53]:
from google.colab import files
uploaded = files.upload()

Saving predictedintent (3).txt to predictedintent (3) (1).txt
Saving predictedslots (2).txt to predictedslots (2) (1).txt


In [54]:
eval("predictedintent (3) (1).txt","predictedslots (2) (1).txt","trueintent.txt","trueslots.txt")

Result on slot filling
accuracy: 0.9719445905663686
F1 on slot filling: 0.9711588303623895
Result on intent classification
accuracy: 0.974
F1 on intent: 0.9741258939955135


Results on Test Set 

In [55]:
from google.colab import files
uploaded = files.upload()

Saving trueintent.txt to trueintent (2).txt
Saving trueslots.txt to trueslots (2).txt


In [62]:
from google.colab import files
uploaded = files.upload()

Saving predictedintent (2).txt to predictedintent (2) (6).txt
Saving predictedslots (1).txt to predictedslots (1) (9).txt


In [63]:
eval("predictedintent (2) (6).txt","predictedslots (1) (9).txt","trueintent (2).txt","trueslots (2).txt")

Result on slot filling
accuracy: 0.9640986468790921
F1 on slot filling: 0.9608960474426598
Result on intent classification
accuracy: 0.93505039193729
F1 on intent: 0.9359771992071622


### Plots

![alt text](https://drive.google.com/uc?id=1fBsLfrwDSpj8_f7bPKVakj4iT3kj7ENN)
![alt text](https://drive.google.com/uc?id=141dQ7YdXES7pmDqsNFOF_RxOof9SiBm_)

## Experiment 17

In this case we we used 20 epochs with 0.0006 learning rate and 8 batch size. This was done in order to observe the affect of weighted loss average on increasing the epochs. Though results increased slightly as comapre to case 11 but results obtained in case 16 were the best.

Results on validation Set

In [58]:
from google.colab import files
uploaded = files.upload()

Saving predictedintent (2).txt to predictedintent (2) (4).txt
Saving predictedslots (1).txt to predictedslots (1) (7).txt


In [59]:
eval("predictedintent (2) (4).txt","predictedslots (1) (7).txt","trueintent (2).txt","trueslots (2).txt")

Result on slot filling
accuracy: 0.9742240925828511
F1 on slot filling: 0.9715903323586719
Result on intent classification
accuracy: 0.956
F1 on intent: 0.9574681161528438


Result on Test Set

In [60]:
from google.colab import files
uploaded = files.upload()

Saving predictedintent (2).txt to predictedintent (2) (5).txt
Saving predictedslots (1).txt to predictedslots (1) (8).txt


In [61]:
eval("predictedintent (2) (5).txt","predictedslots (1) (8).txt","trueintent (2).txt","trueslots (2).txt")

Result on slot filling
accuracy: 0.9599519860323003
F1 on slot filling: 0.953734962100024
Result on intent classification
accuracy: 0.9283314669652856
F1 on intent: 0.928742280705396


### Plots
![alt text](https://drive.google.com/uc?id=1FoILAo9hVIW_i2089McGceoegRoTER5h)
![alt text](https://drive.google.com/uc?id=1eE9ottBexl6OD0ZnAIngtx0czqTUGUXc)

## Training Details

In order to train the model we run the input sequence token by token in encoder. Last hidden state is fed to the decoder as input along with the context vector. Hidden state conatins the information of entire sequence but with a strong focus on the parts surrounding the word at step i. Here we have used cross entropy loss and have experimented with weight distribution to see its affect on the accuracy. 

In [0]:
def train( train_data, encoder, decoder):
    # loss_function_1 = nn.CrossEntropyLoss(ignore_index=0)
    loss_function_1 = nn.NLLLoss(ignore_index=0)
    loss_function_2 = nn.CrossEntropyLoss()
    enc_optim = optim.Adam(encoder.parameters(), lr=learning_rate)
    dec_optim = optim.Adam(decoder.parameters(), lr=learning_rate)
    losses = []
    
    for step in range(step_size):
        losses = []
        for i, batch in enumerate(getBatch(batch_size, train_data)):
            x, y_1, y_2 = zip(*batch)
            x = torch.cat(x)
            tag_target = torch.cat(y_1).view(-1)
            intent_target = torch.cat(y_2)
            x_mask = torch.cat([Variable(torch.BoolTensor(tuple(map(lambda s: s == 0, t.data)))) for t in x])\
                .view(batch_size, -1).cuda()

            encoder.zero_grad()
            decoder.zero_grad()

            output, hidden_c = encoder(x, x_mask)
            start_decode = Variable(torch.LongTensor([[0]*batch_size])).transpose(1, 0)
            # pdb.set_trace()
            tag_score, intent_score = decoder(start_decode, hidden_c, output, x_mask)

            loss_1 = loss_function_1(tag_score, tag_target.cuda())
            loss_2 = loss_function_2(intent_score, intent_target.cuda())

            loss = loss_1+loss_2
            losses.append(loss.item())
            loss.backward()

            torch.nn.utils.clip_grad_norm(encoder.parameters(), 5.0)
            torch.nn.utils.clip_grad_norm(decoder.parameters(), 5.0)

            enc_optim.step()
            dec_optim.step()

            

            if i % 10 == 0:
                print("epoch", step, " step", i, " : ", np.mean(losses))
                losses.append(losses)
                


    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    
    # pdb.set_trace()
    torch.save(decoder.state_dict(), os.path.join(model_dir, 'jointnlu-decoder.pkl'))
    torch.save(encoder.state_dict(), os.path.join(model_dir, 'jointnlu-encoder.pkl'))
    print("Train Complete!")